This a simple baseline using the LGBM algorithm. This is a small modification to https://www.kaggle.com/lgreig/simple-lgbm-baseline.


In [ ]:
import riiideducation
# import dask.dataframe as dd
import pandas as pd
import numpy as np
env = riiideducation.make_env()

In [ ]:
#note I also import the content_type_id to determine if I should filter out negative answered correctly
train= pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                usecols=[1, 2, 3, 4, 7], dtype={'timestamp': 'int64', 'user_id': 'int64' ,'content_id': 'int16', 'content_type_id': 'bool', 'answered_correctly':'int8'}
              )

In [ ]:
train.head()

Affirmatives (True) are only for those with a different type of content. Probably not real questions.

In [ ]:
train = train[train.content_type_id == False]

In [ ]:
train.info()

In [ ]:
#Make a simple tree model and estimate score

In [ ]:
#arrange by timestamp

train = train.sort_values(['timestamp'], ascending=True).reset_index(drop = True)
train.head()

In [ ]:
train.tail()

In [ ]:
train.drop(['timestamp', 'content_type_id'], axis=1,   inplace=True)

In [ ]:
len(list(set(train.iloc[0:90000000,:]['user_id']).intersection(set(train.iloc[90000000:99000000,:]['user_id']))))

In [ ]:
results_c = train.iloc[0:90000000,:][['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c.columns = ["answered_correctly_content"]
results_c.head()

In [ ]:
results_u = train.iloc[0:90000000,:][['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum'])
results_u.columns = ["answered_correctly_user", 'sum']
results_u.head()

In [ ]:
X = train.iloc[90000000:99271299,:]


In [ ]:
X = pd.merge(X, results_u, on=['user_id'], how="left")
X = pd.merge(X, results_c, on=['content_id'], how="left")

In [ ]:
X=X.sort_values(['user_id'])


In [ ]:
Y = X[["answered_correctly"]]
X = X.drop(["answered_correctly"], axis=1)
X.head()

In [ ]:
Y.head()

In [ ]:
X = X[['answered_correctly_user', 'answered_correctly_content', 'sum']] 
X['answered_correctly_user'].fillna(0.5,  inplace=True)
X['answered_correctly_content'].fillna(0.5,  inplace=True)
X['sum'].fillna(0, inplace = True)

In [ ]:
import lightgbm as lgb

model = lgb.LGBMClassifier(num_leaves = 46, learning_rate = 0.11436513141203779, 
                           subsample_for_bin = 130000, min_child_samples = 470, 
                           reg_alpha = 0.5, reg_lambda = 0.26, subsample = 0.5, 
                           is_unbalance = False, n_estimators = 1000, 
                           objective = 'binary', random_state = 126)

model.fit(X, Y)

In [ ]:
y_pred = model.predict_proba(X)[:, 1]
y_true = np.array(Y)

In [ ]:
Y.value_counts()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred)

In [ ]:
###Make sure it works on the test set

In [ ]:
test =  pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')
test.shape

In [ ]:
test = pd.merge(test, results_u, on=['user_id'],  how="left")
test = pd.merge(test, results_c, on=['content_id'],  how="left")

In [ ]:
test.shape

In [ ]:
test[['answered_correctly_user', 'answered_correctly_content', 'sum']]
test['answered_correctly_user'].fillna(0.5, inplace=True)
test['answered_correctly_content'].fillna(0.5, inplace=True)
test['sum'].fillna(0, inplace = True)

In [ ]:
y_pred = model.predict_proba(test[['answered_correctly_user', 'answered_correctly_content', 'sum']])[:, 1]

In [ ]:
y_pred.shape

In [ ]:
test[["answered_correctly"]] = y_pred
test.head()

In [ ]:
#################

In [ ]:
#Make preds

results_c = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c.columns = ["answered_correctly_content"]

results_u = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum'])
results_u.columns = ["answered_correctly_user", 'sum']

In [ ]:
results_c.head()

In [ ]:
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = pd.merge(test_df, results_u, on=['user_id'],  how="left")
    test_df = pd.merge(test_df, results_c, on=['content_id'],  how="left")
    test_df['answered_correctly_user'].fillna(0.5, inplace=True)
    test_df['answered_correctly_content'].fillna(0.5, inplace=True)
    test_df['sum'].fillna(0, inplace=True)
    test_df['answered_correctly'] =  model.predict_proba(test_df[['answered_correctly_user', 'answered_correctly_content', 'sum']])[:, 1]
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])